# Barclays Vortex Analysis
This notebok runs analysis on the Barcalys Vortex (new3) strategy.

## Import Libraries

In [ ]:
import os
os.chdir('..')

import plotly.express as px
from EquityHedging.datamanager import data_manager as dm
from EquityHedging.analytics import summary
from EquityHedging.reporting.excel import reports as rp
from EquityHedging.reporting import formatter as fmt, plots
from ipywidgets import interact, interact_manual
import pandas as pd
import plotly.graph_objects as go


## Import Equity Benchmark & Vortex Data

In [ ]:
equity_bmk = 'SPTR'
strat_drop_list = ['99%/90% Put Spread','Down Var', 'VOLA', 'Dynamic Put Spread', 'VRR',
       'GW Dispersion', 'Corr Hedge', 'Def Var']
include_fi = False
#create returns data dictionary for equity benchmark
returns= dm.get_equity_hedge_returns(equity_bmk, include_fi, strat_drop_list)

## Import new3 robust analysis data

In [ ]:
new_3_robust= pd.read_excel(dm.NEW_DATA+'Barclays_new3_robustness.xlsx',
                                           sheet_name = 'data', index_col=0)

## Create dictionary to slice data into params

In [ ]:
col_dict = {'max_vix':['new3', 'maxVIXDelta=0.65', 'maxVIXDelta=0.7','maxVIXDelta=0.75', 
                       'maxVIXDelta=0.8', 'maxVIXDelta=0.85','maxVIXDelta=0.9', 'maxVIXDelta=1.1'],
           'short_put_strike': ['new3','shortPutStrikes=-0.35,-0.3,-0.25', 
                                'shortPutStrikes=-0.3,-0.25,-0.2','shortPutStrikes=-0.2', 'shortPutStrikes=-0.25',
                                'shortPutStrikes=-0.3','shortPutStrikes=-0.35', 'shortPutStrikes=-0.4'],
           'long_put_strike':['new3', 'longSPXStrike=-0.1','longSPXStrike=-0.15'],
           'vix_strike':['new3', 'VIXStrike=0.2', 'VIXStrike=0.3'],
           'roll_dates':['new3', 'rollDates=-20,-15,-10,-5', 'rollDates=-15,-10,-5,0', 
                         'rollDates=-10,-5,0,5', 'rollDates=0,5,10,15', 'rollDates=5,10,15,20']}

#slice into diff params
params_dict = {}
for key in col_dict:
    temp_df = new_3_robust.copy()
    temp_df = temp_df[col_dict[key]]
    params_dict[key] = temp_df.copy()

# Max VIX Delta Parameter

## Display Historical Performance

In [ ]:
params_df = params_dict['max_vix'].copy()

@interact
def display_performance():
    fig = go.Figure()
    for strat in params_df.columns:
        fig.add_trace(go.Scatter(x = pd.to_datetime(params_df.index, format = '%m/%d/%Y'),
                                 y = params_df[strat],mode = 'lines',name = strat))
    fig.update_layout(showlegend=True,plot_bgcolor='White')
    fig.update_xaxes(showline=True,linecolor='black')
    fig.update_yaxes(showline=True,linecolor='black')
    fig.show()

## Compute Max VIX Delta Analytics (Return Stats and Hedge Metrics)

In [ ]:
#create returns data dictionary for strategy
max_vix_dict = dm.get_data_dict(params_dict['max_vix'])

#merge dictionaries
max_vix_dict = dm.merge_dicts(returns,max_vix_dict)

#compute analytics
analytics_freq_list = ['Weekly', 'Monthly']
max_vix_analytics_dict = summary.get_analytics_data(max_vix_dict)

## Display Max VIX Delta Analytics

In [ ]:
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = max_vix_analytics_dict[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

# Short Put Strike Parameter

## Display Historical Performance

In [ ]:
params_df =params_dict['short_put_strike'].copy()

@interact
def display_performance():
    fig = go.Figure()
    for strat in params_df.columns:
        fig.add_trace(go.Scatter(x = pd.to_datetime(params_df.index, format = '%m/%d/%Y'),
                                 y = params_df[strat],mode = 'lines',name = strat))
    fig.update_layout(showlegend=True,plot_bgcolor='White')
    fig.update_xaxes(showline=True,linecolor='black')
    fig.update_yaxes(showline=True,linecolor='black')
    fig.show()

## Compute Short Put Strike Analytics

In [ ]:
#create returns data dictionary for strategy
sp_dict = dm.get_data_dict(params_dict['short_put_strike'])

#merge dictionaries
sp_dict = dm.merge_dicts(returns,sp_dict)

#compute analytics
analytics_freq_list = ['Weekly', 'Monthly']
sp_analytics_dict = summary.get_analytics_data(sp_dict)


## Display Short Put Strike Analytics

In [ ]:
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = sp_analytics_dict[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

# Long Put Strike Analytics

## Display Historical Performance

In [ ]:
params_df =params_dict['long_put_strike'].copy()

@interact
def display_performance():
    fig = go.Figure()
    for strat in params_df.columns:
        fig.add_trace(go.Scatter(x = pd.to_datetime(params_df.index, format = '%m/%d/%Y'),
                                 y = params_df[strat],mode = 'lines',name = strat))
    fig.update_layout(showlegend=True,plot_bgcolor='White')
    fig.update_xaxes(showline=True,linecolor='black')
    fig.update_yaxes(showline=True,linecolor='black')
    fig.show()

## Compute Long Put Strike Analytics

In [ ]:
#create returns data dictionary for strategy
lp_dict = dm.get_data_dict(params_dict['long_put_strike'])

#merge dictionaries
lp_dict = dm.merge_dicts(returns,lp_dict)

#compute analytics
analytics_freq_list = ['Weekly', 'Monthly']
lp_analytics_dict = summary.get_analytics_data(lp_dict)


## Display Long Put Strike Analytics

In [ ]:
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = lp_analytics_dict[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

# VIX Strike Parameter

## Display Historical Performance

In [ ]:
params_df =params_dict['vix_strike'].copy()

@interact
def display_performance():
    fig = go.Figure()
    for strat in params_df.columns:
        fig.add_trace(go.Scatter(x = pd.to_datetime(params_df.index, format = '%m/%d/%Y'),
                                 y = params_df[strat],mode = 'lines',name = strat))
    fig.update_layout(showlegend=True,plot_bgcolor='White')
    fig.update_xaxes(showline=True,linecolor='black')
    fig.update_yaxes(showline=True,linecolor='black')
    fig.show()

## Compute VIX Strike Analytics

In [ ]:
#create returns data dictionary for strategy
vix_k_dict = dm.get_data_dict(params_dict['vix_strike'])

#merge dictionaries
vix_k_dict = dm.merge_dicts(returns,vix_k_dict)

#compute analytics
analytics_freq_list = ['Weekly', 'Monthly']
vix_k_analytics_dict = summary.get_analytics_data(vix_k_dict)

## Display VIX Strike Analytics

In [ ]:
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = vix_k_analytics_dict[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

# Roll Dates Parameter

## Display Historical Performance

In [ ]:
params_df =params_dict['roll_dates'].copy()

@interact
def display_performance():
    fig = go.Figure()
    for strat in params_df.columns:
        fig.add_trace(go.Scatter(x = pd.to_datetime(params_df.index, format = '%m/%d/%Y'),
                                 y = params_df[strat],mode = 'lines',name = strat))
    fig.update_layout(showlegend=True,plot_bgcolor='White')
    fig.update_xaxes(showline=True,linecolor='black')
    fig.update_yaxes(showline=True,linecolor='black')
    fig.show()

## Compute Roll Dates Analytics

In [ ]:
#create returns data dictionary for strategy
rd_dict = dm.get_data_dict(params_dict['roll_dates'])

#merge dictionaries
rd_dict = dm.merge_dicts(returns,rd_dict)

#compute analytics
analytics_freq_list = ['Weekly', 'Monthly']
rd_analytics_dict = summary.get_analytics_data(rd_dict)

## Display Roll Dates Analytics

In [ ]:
@interact
def display_analytics(frequency=analytics_freq_list, stats=['return_stats', 'hedge_metrics']):
    data = vix_k_analytics_dict[frequency]
    analytics = data[False]
    if stats == 'return_stats':
        return fmt.get_analytics_styler(analytics)
    if stats == 'hedge_metrics':
        if frequency == 'Weekly':
            return fmt.get_analytics_styler(analytics, stats, '1W')
        else:
            return fmt.get_analytics_styler(analytics, stats)

## Compute Historical Sell Offs then slice for different parameters

In [ ]:
#create returns data dictionary for strategy
new_data_dict = dm.get_data_dict(new_3_robust)

#merge dictionaries
ret_data_dict = dm.merge_dicts(returns,new_data_dict)

#compute historical sell offs
hist_dict = summary.get_hist_data(ret_data_dict)

#slice for diff params
hist_list = list(hist_dict[False].columns)[0:2]
for key in col_dict:
    temp_list = hist_list + col_dict[key]
    temp_hs = hist_dict[False].copy()
    temp_hs = temp_hs[temp_list]
    hist_dict[key] = temp_hs

## Display Historical Sell Offs

In [ ]:
@interact
def display_selloffs(params = [False]+list(col_dict.keys())):
    df_hist = hist_dict[params]
    return fmt.get_hist_styler(df_hist)

# Create Report

Run this code below to export analysis into excel spreadsheet
* **strat_report (string)**: provide a name for the excel file
* **selloffs (boolean)**:
    * **True**: if you want historical selloff data in the spreadsheet
    * **False**: if you do not want historical selloff data in the spreadsheet

In [ ]:
strat_report_1 = 'new_3_robustness'
selloffs = True
rp.generate_strat_report(strat_report_1, ret_data_dict, selloffs)